In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
from line_search import *

# Test Line Functions

In [4]:
N = 20
A = np.random.randn(N, N)

In [31]:
def xsquare():
    def f(x):
        return np.dot(x,x)
    def df(x):
        return 2*x
    return f, df

def xAx_plus1():
    def f(x):
        return np.dot(x, np.dot(A, x)) + 1
    def df(x):
        return np.dot(A + A.T, x)
    return f, df

line_funcs = [xsquare, xAx_plus1]

In [102]:
def test_line_search(line_search_algo, assert_algo_conditions_f, test_armijo=True, use_wolfe=True, use_strong_wolfe=True):
    for line_func in line_funcs:
        name = line_func.__name__
        f, df = line_func()
        for k in range(9):
            x = np.random.randn(N) # current point
            p = np.random.randn(N) # direction vector
            if np.dot(p, df(x)) >= 0:
                # skip bec. this is not a descent direction
                continue
        (alpha, fval, dfval), num_fcalls, num_dfcalls = line_search_algo(f, df, x, p, \
                                use_wolfe=use_wolfe, use_strong_wolfe=use_strong_wolfe)
        if not alpha:
            print("Failed to converge, no alpha")
            return False
        if not np.allclose(fval, f(x + alpha*p)):
            print("not all close")
            return False
        return assert_algo_conditions_f(x, p, alpha, f, df, test_armijo=test_armijo, use_wolfe=use_wolfe, use_strong_wolfe=use_strong_wolfe)
        

# Algorithm Specific checks

In [103]:
def assert_strong_wolfe(x, p, alpha, f, df, c1=DEFAULT_C1, c2=DEFAULT_C2, test_armijo=True, use_wolfe=True, use_strong_wolfe=True):
    phik = f(x + alpha * p)
    phi0 = f(x)
    dphik = np.dot(df(x + alpha * p), p)
    dphi0 = np.dot(df(x), p)
    if test_armijo and not phik <= phi0 + c1 * alpha * dphik:
        print("failed armijo condition")
        return False
    if use_wolfe:
        if not dphik >= c2 * dphi0:
            print("failed curvature condition")
            return False
    if use_strong_wolfe:
        if not dphik <= -c2 * dphi0:
            print("failed strong wolfe condition")
            return False
    return True

# Test Backtracking

In [104]:
num_converged = 0
iters = 100
for i in range(iters): 
    if test_line_search(backtracking_linesearch, assert_strong_wolfe, test_armijo=True, \
                        use_wolfe=True, use_strong_wolfe=True):
        num_converged += 1
print("percent_converged: %.4f, %d, %d" % (float(num_converged)/iters, num_converged, iters))

Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to conv

# Test Interpolating

In [105]:
num_converged = 0
iters = 100
for i in range(iters):
    if test_line_search(interpolating_line_search, assert_strong_wolfe, test_armijo=True, \
                        use_wolfe=False, use_strong_wolfe=False):
        num_converged += 1
print("percent_converged: %.4f, %d, %d" % (float(num_converged)/iters, num_converged, iters))

percent_converged: 1.0000, 100, 100


# Test Strong Wolfe with Zoom

In [114]:
num_converged = 0
iters = 100
for i in range(iters):
    if test_line_search(strong_wolfe_with_zoom, assert_strong_wolfe, test_armijo=True, \
                        use_wolfe=True, use_strong_wolfe=True):
        num_converged += 1
print("percent_converged: %.4f, %d, %d" % (float(num_converged)/iters, num_converged, iters))

Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't find appropriate value in zoom
failed armijo condition
Didn't f

# Test Scipy Wolfe

In [107]:
from scipy.optimize.linesearch import line_search_wolfe2

In [110]:
def test_line_search_scipy(line_search_algo, assert_algo_conditions_f, test_armijo=True, \
                           use_wolfe=True, use_strong_wolfe=False):
    for line_func in line_funcs:
        name = line_func.__name__
        f, df = line_func()
        for k in range(9):
            x = np.random.randn(N) # current point
            p = np.random.randn(N) # direction vector 
            if np.dot(p, df(x)) >= 0:
                # skip bec. this is not a descent direction
                continue
        # alpha_star, fc[0], gc[0], phi_star, old_fval, derphi_star
        alpha, func_call, g_call, fval, old_fval, df_val = line_search_algo(f, df, x, p)
        if not alpha:
            print("Failed to converge, no alpha")
            return False
        assert np.allclose(fval, f(x + alpha*p))
        assert_algo_conditions_f(x, p, alpha, f, df, test_armijo=test_armijo, use_wolfe=use_wolfe, use_strong_wolfe=use_strong_wolfe)
        return True

In [113]:
num_converged = 0
iters = 100
for i in range(iters):
    if test_line_search_scipy(line_search_wolfe2, assert_strong_wolfe, test_armijo=True, \
                              use_wolfe=True, use_strong_wolfe=True):
        num_converged += 1
print("percent_converged: %.4f, %d, %d" % (float(num_converged)/iters, num_converged, iters))

Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to converge, no alpha
Failed to conv

/Users/timwee/anaconda3/lib/python3.5/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
